# 03 Backtest
Backtest the strategy.

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import pandas as pd
from core.data_loader import load_price_csv, align_pairs
from core.coint import engle_granger
from core.signal import zscore, mean_reversion_signals, compute_spread
from core.backtester import backtest_spread_strategy
from core.metrics import summarize_performance

In [ ]:
df = load_price_csv("../data/prices.csv", date_col="date")
df_pairs = align_pairs(df, ["AAA", "BBB"])
y = df_pairs["BBB"]
x = df_pairs["AAA"]

eg_res = engle_granger(y, x)
spread = compute_spread(y, x, eg_res.hedge_ratio, eg_res.intercept)
z = zscore(spread, window=30)
signals = mean_reversion_signals(z, entry_z=1.5, exit_z=0.5)

In [ ]:
bt_res = backtest_spread_strategy(spread, signals)
summary = summarize_performance(bt_res.returns, bt_res.turnover)
print(summary)

In [ ]:
(1 + bt_res.returns).cumprod().plot(title="Equity Curve")